# 📊 RAG_Standard 평가 시스템 탐방

이 노트북에서는 RAG 시스템의 답변 품질을 평가하는 방법을 배웁니다.

## 학습 내용
1. 평가 시스템 개요
2. 실시간 품질 점수 확인
3. 피드백을 통한 품질 개선
4. 배치 평가 실행 (관리자)

## 평가 지표
- **Faithfulness (충실도)**: 답변이 검색된 문서에 근거하는가?
- **Relevance (관련성)**: 답변이 질문 의도에 부합하는가?

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import requests

BASE_URL = "http://localhost:8000"

# API Key 설정
# .env 파일의 FASTAPI_AUTH_KEY 값을 입력하세요
API_KEY = ""  # 예: "your-secret-key"

def api_call(method, endpoint, auth=False, admin=False, **kwargs):
    """API 호출 헬퍼

    Args:
        auth: True면 X-API-Key 헤더 추가 (일반 API용)
        admin: True면 관리자 API용 인증 (항상 헤더 추가)
    """
    url = f"{BASE_URL}{endpoint}"
    headers = kwargs.pop('headers', {})
    if (auth or admin) and API_KEY:
        headers['X-API-Key'] = API_KEY
    return requests.request(method, url, headers=headers, timeout=30, **kwargs)

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'AppleGothic'  # Mac
# plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False

# 연결 확인
response = requests.get(f"{BASE_URL}/health", timeout=5)
if response.status_code == 200:
    print("✅ RAG 서버 연결 성공!")
    if not API_KEY:
        print("ℹ️ API_KEY가 설정되지 않았습니다. 개발 환경에서만 동작합니다.")

## 1️⃣ 실시간 품질 점수 확인

RAG 시스템은 Self-RAG 기능으로 답변 생성 시 품질 점수를 함께 반환합니다.

In [ ]:
# 채팅 요청 및 품질 점수 확인
questions = [
    "RAG_Standard 설치 방법은?",
    "하이브리드 검색이 뭐야?",
    "DI 컨테이너의 장점은?",
]

results = []
for q in questions:
    response = api_call("POST", "/api/chat", auth=True, json={"message": q})
    if response.status_code == 200:
        data = response.json()
        quality = data.get('metadata', {}).get('quality', {})
        results.append({
            '질문': q[:20] + '...',
            '품질 점수': quality.get('score', 0),
            '신뢰도': quality.get('confidence', 'N/A'),
            'Self-RAG': '✅' if quality.get('self_rag_applied') else '❌'
        })
    elif response.status_code == 401:
        print("❌ 인증 실패: API_KEY를 설정하세요")
        break

if results:
    df = pd.DataFrame(results)
    print(df.to_string(index=False))
else:
    print("결과가 없습니다. API_KEY를 확인하세요.")

In [ ]:
# 품질 점수 시각화
if results:
    fig, ax = plt.subplots(figsize=(10, 5))

    questions_short = [r['질문'] for r in results]
    scores = [r['품질 점수'] for r in results]

    bars = ax.barh(questions_short, scores, color='steelblue')
    ax.set_xlabel('품질 점수')
    ax.set_title('질문별 답변 품질 점수')
    ax.set_xlim(0, 1)

    # 점수 표시
    for bar, score in zip(bars, scores, strict=False):
        ax.text(score + 0.02, bar.get_y() + bar.get_height()/2,
                f'{score:.2f}', va='center')

    plt.tight_layout()
    plt.show()

## 2️⃣ 피드백을 통한 품질 개선

사용자 피드백은 Golden Dataset 구축에 활용됩니다.

### 피드백 워크플로우
1. 사용자가 답변에 👍/👎 피드백
2. 좋은 피드백은 Golden Dataset 후보로 등록
3. 관리자가 검토하여 정답셋에 추가
4. 정답셋으로 시스템 성능 벤치마크

In [ ]:
# 피드백 제출 시뮬레이션
feedback_examples = [
    {"rating": 1, "comment": "정확하고 도움이 됨"},
    {"rating": -1, "comment": "질문과 관련 없는 답변"},
    {"rating": 1, "comment": "상세한 설명 감사"},
]

print("📝 피드백 예시:\n")
for i, fb in enumerate(feedback_examples, 1):
    emoji = "👍" if fb['rating'] == 1 else "👎"
    print(f"{i}. {emoji} {fb['comment']}")

print("\n피드백은 POST /api/chat/feedback API로 제출합니다.")

## 3️⃣ 배치 평가 (관리자)

⚠️ 관리자 API 키가 필요합니다.

배치 평가는 여러 질문-답변 쌍을 한번에 평가합니다.

In [ ]:
# 배치 평가 요청 (관리자 전용)
eval_data = {
    "dataset": [
        {
            "query": "RAG_Standard 설치 방법은?",
            "response": "git clone으로 저장소를 클론하고 uv sync로 의존성을 설치합니다.",
            "context": "Quickstart: git clone, uv sync, make quickstart"
        },
        {
            "query": "하이브리드 검색이란?",
            "response": "Dense(의미) 검색과 Sparse(BM25) 검색을 결합한 방식입니다.",
            "context": "Weaviate: Dense + Sparse 하이브리드 검색"
        }
    ],
    "metrics": ["faithfulness", "relevance"]
}

if not API_KEY:
    print("❌ 관리자 API 호출을 위해 API_KEY를 먼저 설정하세요")
else:
    response = api_call("POST", "/api/admin/evaluate", admin=True, json=eval_data)

    if response.status_code == 200:
        results = response.json()
        print("📊 배치 평가 결과:\n")
        for i, r in enumerate(results.get('results', []), 1):
            print(f"항목 {i}:")
            print(f"  - Faithfulness: {r.get('faithfulness', 0):.2f}")
            print(f"  - Relevance: {r.get('relevance', 0):.2f}")
            print()
    elif response.status_code == 401:
        print("❌ 인증 실패: API_KEY를 확인하세요")
    elif response.status_code == 404:
        print("ℹ️ 배치 평가 API가 비활성화되어 있습니다.")
    else:
        print(f"❌ 오류: {response.status_code}")

## 📖 평가 지표 상세 설명

### Internal Metrics (자체 지표)
| 지표 | 설명 | 측정 방식 |
|------|------|----------|
| Faithfulness | 답변이 검색된 문서에 근거하는가 | 문장별 출처 확인 |
| Relevance | 답변이 질문 의도에 부합하는가 | 의미 유사도 계산 |

### Ragas Metrics (외부 표준 지표)
| 지표 | 설명 |
|------|------|
| Context Precision | 관련 문서가 상단에 위치하는가 |
| Answer Semantic Similarity | 정답과의 의미적 유사도 |

### 점수 해석
- **0.8 이상**: 우수 (신뢰할 수 있음)
- **0.6 ~ 0.8**: 양호 (대체로 정확)
- **0.6 미만**: 개선 필요 (검증 권장)

## 🎉 완료!

RAG 평가 시스템의 핵심 개념을 익혔습니다.

### 추가 리소스
- **[평가 시스템 문서](../docs/EVALUATION_SYSTEM.md)**: 상세 아키텍처
- **CLI 평가**: `make eval` (로컬에서 배치 평가)

### 실습 과제
1. 다양한 질문으로 품질 점수 비교해보기
2. 피드백 API를 사용해 데이터 수집해보기
3. 품질 점수와 실제 답변 품질 상관관계 분석